In [ ]:
! pip install datasets transformers seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
import transformers

print(transformers.__version__)

4.22.1


In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "bert-base-chinese"
batch_size = 16

# loading the dataset

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Adapting/chinese_biomedical_NER_dataset")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    dev: Dataset({
        features: ['sequences', 'tags', 'tag_ids', 'id'],
        num_rows: 41
    })
    train: Dataset({
        features: ['sequences', 'tags', 'tag_ids', 'id'],
        num_rows: 914
    })
    test: Dataset({
        features: ['sequences', 'tags', 'tag_ids', 'id'],
        num_rows: 41
    })
})

In [ ]:
dataset["train"][0]

{'sequences': [',',
  '患',
  '者',
  '3',
  '月',
  '前',
  '因',
  '“',
  '直',
  '肠',
  '癌',
  '”',
  '于',
  '在',
  '我',
  '院',
  '于',
  '全',
  '麻',
  '上',
  '行',
  '直',
  '肠',
  '癌',
  '根',
  '治',
  '术',
  '(',
  'd',
  'i',
  'x',
  'o',
  'n',
  '术',
  ')',
  ',',
  '手',
  '术',
  '过',
  '程',
  '顺',
  '利',
  ',',
  '术',
  '后',
  '给',
  '予',
  '抗',
  '感',
  '染',
  '及',
  '营',
  '养',
  '支',
  '持',
  '治',
  '疗',
  ',',
  '患',
  '者',
  '恢',
  '复',
  '好',
  ',',
  '切',
  '口',
  '愈',
  '合',
  '良',
  '好',
  '。',
  ',',
  '术',
  '后',
  '病',
  '理',
  '示',
  ':',
  '直',
  '肠',
  '腺',
  '癌',
  '(',
  '中',
  '低',
  '度',
  '分',
  '化',
  ')',
  ',',
  '浸',
  '润',
  '溃',
  '疡',
  '型',
  ',',
  '面',
  '积',
  '3',
  '.',
  '5',
  '*',
  '2',
  'c',
  'm',
  ',',
  '侵',
  '达',
  '外',
  '膜',
  '。',
  '双',
  '端',
  '切',
  '线',
  '另',
  '送',
  '“',
  '近',
  '端',
  '”',
  '、',
  '“',
  '远',
  '端',
  '”',
  '及',
  '环',
  '周',
  '底',
  '部',
  '切',
  '除',
  '面',
  '未',
  '查',
  '见',
  '癌',
  '。',
  '肠',
  '壁',


In [ ]:
tag_set = [
 'B_手术',
 'I_疾病和诊断',
 'B_症状',
 'I_解剖部位',
 'I_药物',
 'B_影像检查',
 'B_药物',
 'B_疾病和诊断',
 'I_影像检查',
 'I_手术',
 'B_解剖部位',
 'O',
 'B_实验室检验',
 'I_症状',
 'I_实验室检验'
 ]
 
tag2id = lambda tag: tag_set.index(tag)
id2tag = lambda id: tag_set[id]


# Preprocessing the data
the raw data in this dataset are in form of `string`, so we need to convert these `string` objects to `list` objects first.

In [ ]:
import ast
x = '[ "A","B","C" , " D"]'
x = ast.literal_eval(x)
x

['A', 'B', 'C', ' D']

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
tokenizer('你知道感冒病毒吗?')

{'input_ids': [101, 872, 4761, 6887, 2697, 1088, 4567, 3681, 1408, 136, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer('你')

{'input_ids': [101, 872, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [ ]:
example = dataset["train"][1]
print(example["sequences"])

[',', '患', '者', '因', '罹', '患', '“', '胃', '癌', '”', '于', '2', '0', '1', '3', '-', '1', '0', '-', '2', '9', '在', '我', '院', '予', '行', '全', '麻', '上', '胃', '癌', '根', '治', '术', ',', ',', '术', '中', '见', ':', '腹', '腔', '内', '腹', '水', ',', '腹', '膜', '无', '转', '移', ',', '肝', '脏', '未', '触', '及', '明', '显', '转', '移', '性', '灶', ',', '肿', '瘤', '位', '于', '胃', '体', '、', '胃', '底', '部', ',', '小', '弯', '侧', '偏', '后', '壁', ',', '约', '5', '*', '4', '*', '2', 'c', 'm', '大', '小', ',', '肿', '瘤', '已', '侵', '达', '浆', '膜', '外', ',', '第', '1', '、', '3', '组', '淋', '巴', '结', '肿', '大', ',', '肿', '瘤', '尚', '能', '活', '动', ',', '经', '探', '查', '决', '定', '行', '全', '胃', '切', '除', ',', '空', '肠', 'j', '字', '代', '胃', '术', '。', '手', '术', '顺', '利', ',', '术', '后', '积', '极', '予', '相', '关', '对', '症', '支', '持', '治', '疗', ';', ',', '后', '病', '理', '示', ':', '胃', '底', '、', '体', '小', '弯', '侧', '低', '分', '化', '腺', '癌', ',', '部', '分', '为', '印', '戒', '细', '胞', '癌', '图', '像', ',', '蕈', '伞', '型', ',', '面', '积', '5', '.', '2', '*', '3', '.',

In [ ]:
tokenized_input = tokenizer(example["sequences"], is_split_into_words=True,truncation=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', ',', '患', '者', '因', '罹', '患', '[UNK]', '胃', '癌', '[UNK]', '于', '2', '0', '1', '3', '-', '1', '0', '-', '2', '9', '在', '我', '院', '予', '行', '全', '麻', '上', '胃', '癌', '根', '治', '术', ',', ',', '术', '中', '见', ':', '腹', '腔', '内', '腹', '水', ',', '腹', '膜', '无', '转', '移', ',', '肝', '脏', '未', '触', '及', '明', '显', '转', '移', '性', '灶', ',', '肿', '瘤', '位', '于', '胃', '体', '、', '胃', '底', '部', ',', '小', '弯', '侧', '偏', '后', '壁', ',', '约', '5', '*', '4', '*', '2', 'c', 'm', '大', '小', ',', '肿', '瘤', '已', '侵', '达', '浆', '膜', '外', ',', '第', '1', '、', '3', '组', '淋', '巴', '结', '肿', '大', ',', '肿', '瘤', '尚', '能', '活', '动', ',', '经', '探', '查', '决', '定', '行', '全', '胃', '切', '除', ',', '空', '肠', 'j', '字', '代', '胃', '术', '。', '手', '术', '顺', '利', ',', '术', '后', '积', '极', '予', '相', '关', '对', '症', '支', '持', '治', '疗', ';', ',', '后', '病', '理', '示', ':', '胃', '底', '、', '体', '小', '弯', '侧', '低', '分', '化', '腺', '癌', ',', '部', '分', '为', '印', '戒', '细', '胞', '癌', '图', '像', ',', '蕈', '伞', '型', ',', '面', '积', '5', '.', '2

In [ ]:
len(example[f"tag_ids"]), len(tokenized_input["input_ids"])

(567, 512)

In [ ]:
len([-100] + example['tag_ids'] + [-100] )

569

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220

In [ ]:
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(example["sequences"], truncation=True, is_split_into_words=True)
    word_ids = tokenized_inputs.word_ids()

    labels = []

    for i in word_ids:
        if i is None:
            labels.append(-100)
        else:
            labels.append(example['tag_ids'][i])

    tokenized_inputs['labels'] = labels

    return tokenized_inputs

In [ ]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=False,remove_columns=dataset['train'].column_names)

In [ ]:
for j,i in enumerate(tokenized_dataset['train']):
    label = i['labels']
    input = i['input_ids']

    assert label[0] == label[-1] == -100
    assert len(label) == len(input)


In [ ]:
tokenized_dataset

DatasetDict({
    dev: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 41
    })
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 914
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 41
    })
})

In [ ]:
tokenized_dataset['train'][1]

{'input_ids': [101,
  117,
  2642,
  5442,
  1728,
  5395,
  2642,
  100,
  5517,
  4617,
  100,
  754,
  123,
  121,
  122,
  124,
  118,
  122,
  121,
  118,
  123,
  130,
  1762,
  2769,
  7368,
  750,
  6121,
  1059,
  7937,
  677,
  5517,
  4617,
  3418,
  3780,
  3318,
  117,
  117,
  3318,
  704,
  6224,
  131,
  5592,
  5579,
  1079,
  5592,
  3717,
  117,
  5592,
  5606,
  3187,
  6760,
  4919,
  117,
  5498,
  5552,
  3313,
  6239,
  1350,
  3209,
  3227,
  6760,
  4919,
  2595,
  4131,
  117,
  5514,
  4606,
  855,
  754,
  5517,
  860,
  510,
  5517,
  2419,
  6956,
  117,
  2207,
  2482,
  904,
  974,
  1400,
  1880,
  117,
  5276,
  126,
  115,
  125,
  115,
  123,
  145,
  155,
  1920,
  2207,
  117,
  5514,
  4606,
  2347,
  909,
  6809,
  3841,
  5606,
  1912,
  117,
  5018,
  122,
  510,
  124,
  5299,
  3900,
  2349,
  5310,
  5514,
  1920,
  117,
  5514,
  4606,
  2213,
  5543,
  3833,
  1220,
  117,
  5307,
  2968,
  3389,
  1104,
  2137,
  6121,
  1059,
  5517,
  

# Fine-tuning the model

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(tag_set))

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-c

In [ ]:
model_name = model_checkpoint
args = TrainingArguments(
    f"{model_name}-finetuned-NER-biomedical",
    evaluation_strategy = "epoch",
    save_strategy= 'epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=100,
    weight_decay=0.01,
    logging_steps = 50,
    load_best_model_at_end = True,
    metric_for_best_model = 'recall',
    save_total_limit=3,
    push_to_hub=True
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [ ]:
labels = [tag_set[i] for i in example[f"tag_ids"]]
metric.compute(predictions=[labels], references=[labels])

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_疾病和诊断 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_疾病和诊断 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_手术 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_手术 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_解剖部位 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:1

{'_手术': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 '_疾病和诊断': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 '_症状': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 '_解剖部位': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 8},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [tag_set[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [tag_set[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    
)

Cloning https://huggingface.co/Adapting/bert-base-chinese-finetuned-NER-biomedical into local empty directory.


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 914
  Num Epochs = 100
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5800
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.640700,0.163701,0.562500,0.709360,0.627451,0.955163
2,0.178500,0.112219,0.702158,0.801314,0.748466,0.968620
3,0.133900,0.101312,0.730600,0.819376,0.772446,0.970570
4,0.100500,0.097331,0.743777,0.834154,0.786378,0.972834
5,0.090200,0.104213,0.778452,0.842365,0.809148,0.971261


***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_疾病和诊断 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_疾病和诊断 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_症状 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_症状 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_解剖部位 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/loc

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.640700,0.163701,0.562500,0.709360,0.627451,0.955163
2,0.178500,0.112219,0.702158,0.801314,0.748466,0.968620
3,0.133900,0.101312,0.730600,0.819376,0.772446,0.970570
4,0.100500,0.097331,0.743777,0.834154,0.786378,0.972834
5,0.090200,0.104213,0.778452,0.842365,0.809148,0.971261
6,0.080500,0.109667,0.765957,0.827586,0.795580,0.967991
7,0.058600,0.114961,0.761128,0.842365,0.799688,0.969186
8,0.052200,0.116291,0.780488,0.840722,0.809486,0.968998
9,0.051200,0.112643,0.773669,0.858785,0.814008,0.971576
10,0.044800,0.116019,0.788491,0.832512,0.809904,0.969438


***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_疾病和诊断 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_疾病和诊断 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_症状 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_症状 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_解剖部位 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/loc

TrainOutput(global_step=5800, training_loss=0.017403822651711004, metrics={'train_runtime': 5248.4497, 'train_samples_per_second': 17.415, 'train_steps_per_second': 1.105, 'total_flos': 2.386835037080172e+16, 'train_loss': 0.017403822651711004, 'epoch': 100.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 41
  Batch size = 16


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_疾病和诊断 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_疾病和诊断 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_症状 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_症状 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_解剖部位 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:1

{'eval_loss': 0.16049373149871826,
 'eval_precision': 0.804185351270553,
 'eval_recall': 0.8834154351395731,
 'eval_f1': 0.8419405320813771,
 'eval_accuracy': 0.9736511130675387,
 'eval_runtime': 1.0726,
 'eval_samples_per_second': 38.225,
 'eval_steps_per_second': 2.797,
 'epoch': 100.0}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [tag_set[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [tag_set[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

***** Running Prediction *****
  Num examples = 41
  Batch size = 16


{'_实验室检验': {'precision': 0.8333333333333334,
  'recall': 0.967741935483871,
  'f1': 0.8955223880597015,
  'number': 31},
 '_影像检查': {'precision': 0.9310344827586207,
  'recall': 0.9310344827586207,
  'f1': 0.9310344827586207,
  'number': 29},
 '_手术': {'precision': 0.723404255319149,
  'recall': 0.7906976744186046,
  'f1': 0.7555555555555555,
  'number': 43},
 '_疾病和诊断': {'precision': 0.8026315789473685,
  'recall': 0.8591549295774648,
  'f1': 0.8299319727891157,
  'number': 142},
 '_症状': {'precision': 0.975609756097561,
  'recall': 0.9523809523809523,
  'f1': 0.963855421686747,
  'number': 84},
 '_药物': {'precision': 0.9154929577464789,
  'recall': 0.9848484848484849,
  'f1': 0.948905109489051,
  'number': 66},
 '_解剖部位': {'precision': 0.7142857142857143,
  'recall': 0.8411214953271028,
  'f1': 0.7725321888412017,
  'number': 214},
 'overall_precision': 0.804185351270553,
 'overall_recall': 0.8834154351395731,
 'overall_f1': 0.8419405320813771,
 'overall_accuracy': 0.9736511130675387}

In [ ]:
model.push_to_hub('Adapting/bert-base-chinese-finetuned-NER-biomedical')
tokenizer.push_to_hub('Adapting/bert-base-chinese-finetuned-NER-biomedical')

Configuration saved in bert-base-chinese-finetuned-NER-biomedical/config.json
Model weights saved in bert-base-chinese-finetuned-NER-biomedical/pytorch_model.bin
Uploading the following files to Adapting/bert-base-chinese-finetuned-NER-biomedical: config.json,pytorch_model.bin
tokenizer config file saved in bert-base-chinese-finetuned-NER-biomedical/tokenizer_config.json
Special tokens file saved in bert-base-chinese-finetuned-NER-biomedical/special_tokens_map.json
Uploading the following files to Adapting/bert-base-chinese-finetuned-NER-biomedical: special_tokens_map.json,tokenizer.json,tokenizer_config.json,vocab.txt
